# Atelier CQL avancé — Fleet Tracing (IoT) — Étudiant

Durée: **4h**

## Contexte
Vous travaillez sur une plateforme de **Fleet Tracking** (bus/camions). Chaque véhicule (device) envoie des mesures (GPS, vitesse, batterie, température) toutes les quelques secondes.

## Règles Cassandra (à respecter)
- Pas de JOINs
- On part des requêtes (query-first)
- Toute requête doit utiliser la partition key dans le `WHERE`
- Les intervalles de temps se font sur une *clustering column*


## Setup
1. Démarrer l’environnement (racine du repo):

```bash
docker compose up -d --build
```

2. UI CQL: `http://localhost:8889`
3. (Optionnel) Jupyter: `http://localhost:8888` (token = `my-token`)


# Partie 1 — Schéma de base (A)
Créez le keyspace + les tables ci-dessous, puis exécutez-les dans l’UI CQL.

```sql
CREATE KEYSPACE IF NOT EXISTS atelier
WITH replication = {'class':'SimpleStrategy','replication_factor':1};

USE atelier;

CREATE TABLE IF NOT EXISTS devices_by_fleet (
  fleet_id text,
  device_id text,
  model text,
  activated_at timestamp,
  PRIMARY KEY ((fleet_id), device_id)
);

CREATE TABLE IF NOT EXISTS latest_telemetry_by_device (
  device_id text PRIMARY KEY,
  last_ts timestamp,
  lat double,
  lon double,
  speed_kmh double,
  battery_pct int,
  temp_c double
);

CREATE TABLE IF NOT EXISTS telemetry_by_device_day (
  device_id text,
  day date,
  ts timestamp,
  lat double,
  lon double,
  speed_kmh double,
  battery_pct int,
  temp_c double,
  zone text,
  PRIMARY KEY ((device_id, day), ts)
) WITH CLUSTERING ORDER BY (ts DESC);

CREATE TABLE IF NOT EXISTS alerts_by_fleet_day (
  fleet_id text,
  day date,
  severity text,
  ts timestamp,
  device_id text,
  alert_type text,
  message text,
  PRIMARY KEY ((fleet_id, day, severity), ts, device_id)
) WITH CLUSTERING ORDER BY (ts DESC);
```


# Partie 2 — Seed minimal
Insérez quelques données (2 flottes, 3 devices, quelques points de télémétrie, 2 alertes).

Exemples (vous pouvez compléter):

```sql
USE atelier;

INSERT INTO devices_by_fleet (fleet_id, device_id, model, activated_at)
VALUES ('FLEET_PARIS', 'BUS-001', 'GPS-TX-1', '2025-12-01 08:00:00+0000');

INSERT INTO telemetry_by_device_day (device_id, day, ts, lat, lon, speed_kmh, battery_pct, temp_c, zone)
VALUES ('BUS-001', '2025-12-17', '2025-12-17 20:00:00+0000', 48.8566, 2.3522, 32.5, 88, 36.2, 'PARIS-A');

INSERT INTO latest_telemetry_by_device (device_id, last_ts, lat, lon, speed_kmh, battery_pct, temp_c)
VALUES ('BUS-001', '2025-12-17 20:00:00+0000', 48.8566, 2.3522, 32.5, 88, 36.2);
```


# Partie 3 — Exercices requêtes (A)
## Exercice 3.1 — Devices d’une flotte
- Requête: listez les devices de `FLEET_PARIS`.

## Exercice 3.2 — Dernier état (dashboard)
- Requête: récupérez le dernier état de `BUS-001`.

## Exercice 3.3 — Historique par journée (TOP N)
- Requête: les 5 dernières mesures de `BUS-001` le `2025-12-17`.

## Exercice 3.4 — Fenêtre temporelle (range)
- Requête: mesures entre `20:00` et `20:06`.

## Exercice 3.5 — Alertes par flotte/jour/sévérité
- Requête: alertes `HIGH` de `FLEET_PARIS` le `2025-12-17`.


# Partie 4 — Pitfalls (B)
## Pitfall 1 — `ALLOW FILTERING`
1) Créez une table naïve `telemetry_naive` qui contient `fleet_id` comme colonne simple.
2) Tentez une requête par flotte (vous serez tenté d’utiliser `ALLOW FILTERING`).
3) Expliquez pourquoi c’est mauvais à grande échelle.

## Correction attendue
Créer une table dédiée `telemetry_by_fleet_day (fleet_id, day, ts, ...)` + requête sans filtering.

## Pitfall 2 — TTL/tombstones
- Ajoutez un `USING TTL 604800` (7 jours) sur un insert de télémétrie.
- Expliquez l’impact (tombstones) et quand c’est acceptable.
